In [19]:
import pandas as pd
from Master import Master
from autogluon.tabular import TabularDataset, TabularPredictor


pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

In [11]:
target_A = pd.read_parquet("../A/train_targets.parquet").assign(time=lambda df: pd.to_datetime(df['time']))
#test_A=pd.read_parquet("../A/X_test_estimated.parquet")
A_observed=pd.read_parquet("../A/X_train_observed.parquet")
A_estimated=pd.read_parquet("../A/X_train_estimated.parquet").drop(columns=['date_calc'])
target_B = pd.read_parquet("../B/train_targets.parquet").assign(time=lambda df: pd.to_datetime(df['time']))
B_observed = pd.read_parquet("../B/X_train_observed.parquet")
B_estimated=pd.read_parquet("../B/X_train_estimated.parquet").drop(columns=['date_calc'])
target_C = pd.read_parquet("../C/train_targets.parquet").assign(time=lambda df: pd.to_datetime(df['time']))
C_observed=pd.read_parquet("../C/X_train_observed.parquet")
C_estimated=pd.read_parquet("../C/X_train_estimated.parquet").drop(columns=['date_calc'])

# test sets
test_A=pd.read_parquet("../A/X_test_estimated.parquet").drop(columns=['date_calc'])
test_B=pd.read_parquet("../B/X_test_estimated.parquet").drop(columns=['date_calc'])
test_C=pd.read_parquet("../C/X_test_estimated.parquet").drop(columns=['date_calc'])
test_A




,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
0,2023-05-01 00:00:00,4.4,1.286,912.700012,0.0,0.0,1041.199951,0.0,271.700012,0.0,0.0,0.0,0.0,80.699997,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.900024,0.0,0.0,1000.700012,1007.000000,0.0,0.0,80.300003,1013.299988,NaN,0.0,0.0,0.0,0.0,10.520000,-11.185,0.0,273.799988,80.699997,30210.699219,4.0,2.2,3.4,-0.0
1,2023-05-01 00:15:00,4.3,1.287,912.700012,0.0,0.0,1054.800049,0.0,271.700012,0.0,0.0,0.0,0.0,77.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.700012,0.0,0.0,1000.599976,1006.900024,0.0,0.0,80.300003,1013.200012,NaN,0.0,0.0,0.0,0.0,14.203000,-10.825,0.0,273.799988,77.000000,30003.599609,4.0,2.1,3.4,-0.0
2,2023-05-01 00:30:00,4.3,1.287,912.700012,0.0,0.0,1068.300049,0.0,271.600006,0.0,0.0,0.0,0.0,73.099998,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.599976,0.0,0.0,1000.500000,1006.700012,0.0,0.0,80.300003,1013.000000,NaN,0.0,0.0,0.0,0.0,17.868999,-10.360,0.0,273.799988,73.099998,29797.099609,3.9,2.1,3.3,-0.0
3,2023-05-01 00:45:00,4.3,1.287,912.700012,0.0,0.0,1081.900024,0.0,271.600006,0.0,0.0,0.0,0.0,69.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.500000,0.0,0.0,1000.400024,1006.599976,0.0,0.0,80.199997,1012.900024,NaN,0.0,0.0,0.0,0.0,21.514000,-9.794,0.0,273.799988,69.000000,29618.599609,3.9,2.0,3.3,-0.0
4,2023-05-01 01:00:00,4.3,1.287,NaN,0.0,0.0,1095.400024,0.0,271.600006,0.0,0.0,0.0,0.0,64.500000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.299988,0.0,0.0,1000.200012,1006.500000,0.0,0.0,80.199997,1012.799988,NaN,0.0,0.0,0.0,0.0,25.135000,-9.128,0.0,273.799988,64.500000,29507.500000,3.9,2.0,3.3,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,2023-07-03 22:45:00,9.1,1.207,2090.600098,0.0,0.0,1629.400024,0.0,282.700012,0.0,0.0,0.0,0.0,99.300003,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,993.299988,0.0,0.0,980.799988,986.700012,0.0,0.0,82.199997,992.599976,NaN,0.0,0.0,-0.0,0.0,351.855988,-3.282,0.1,285.399994,99.400002,39931.601562,2.1,1.7,-1.3,0.0
2876,2023-07-03 23:00:00,9.1,1.207,2015.800049,0.0,0.0,1537.300049,0.0,282.700012,0.0,0.0,0.0,0.0,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,993.400024,0.0,0.0,980.900024,986.799988,0.0,0.0,82.500000,992.700012,NaN,0.0,0.0,-0.0,0.0,355.317993,-3.469,0.1,285.299988,100.000000,39574.300781,2.0,1.6,-1.3,0.0
2877,2023-07-03 23:15:00,9.1,1.208,2015.699951,0.0,0.0,1471.400024,0.0,282.700012,0.0,0.0,0.0,0.0,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,993.500000,0.0,0.0,981.000000,986.900024,0.0,0.0,82.800003,992.799988,NaN,0.0,0.0,-0.0,0.0,358.785004,-3.556,0.1,285.299988,100.000000,39627.199219,2.0,1.6,-1.2,0.0
2878,2023-07-03 23:30:00,9.0,1.208,2015.599976,0.0,0.0,1405.500000,0.0,282.700012,0.0,0.0,0.0,0.0,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,993.599976,0.0,0.0,981.000000,986.900024,0.0,0.0,83.199997,992.900024,NaN,0.0,0.0,-0.0,0.0,2.253000,-3.541,0.1,285.200012,100.000000,39691.800781,2.0,1.6,-1.1,0.0


In [12]:
time_col = "date_forecast"
print("test_A.shape)",test_A.shape)
df = test_A.copy()
df[time_col] = pd.to_datetime(df['date_forecast'])

df = df.groupby(pd.Grouper(key=time_col, freq="1H")).mean()
all_nan_mask = df.isnull().all(axis=1)
df = df[~all_nan_mask]
df = df.reset_index()



print("number of days" , df[time_col].dt.date.nunique())

print(df[time_col].dt.date.unique())
print(df.shape)
df.head(30)


test_A.shape) (2880, 46)
number of days 30
[datetime.date(2023, 5, 1) datetime.date(2023, 5, 6)
 datetime.date(2023, 5, 14) datetime.date(2023, 5, 18)
 datetime.date(2023, 5, 20) datetime.date(2023, 5, 21)
 datetime.date(2023, 5, 22) datetime.date(2023, 5, 23)
 datetime.date(2023, 5, 25) datetime.date(2023, 5, 27)
 datetime.date(2023, 5, 28) datetime.date(2023, 5, 30)
 datetime.date(2023, 6, 1) datetime.date(2023, 6, 2)
 datetime.date(2023, 6, 3) datetime.date(2023, 6, 4)
 datetime.date(2023, 6, 6) datetime.date(2023, 6, 9)
 datetime.date(2023, 6, 11) datetime.date(2023, 6, 13)
 datetime.date(2023, 6, 14) datetime.date(2023, 6, 18)
 datetime.date(2023, 6, 19) datetime.date(2023, 6, 22)
 datetime.date(2023, 6, 26) datetime.date(2023, 6, 28)
 datetime.date(2023, 6, 29) datetime.date(2023, 6, 30)
 datetime.date(2023, 7, 2) datetime.date(2023, 7, 3)]
(720, 46)


,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
0,2023-05-01 00:00:00,4.325,1.28675,912.700012,0.000000e+00,0.000000,1061.550049,0.0,271.650024,0.000000,0.000000,0.000000,0.000000e+00,74.949997,6.0,0.000,0.000,0.000,0.000,0.000,0.00,1.00,1013.674988,0.0,0.0,1000.549988,1006.799988,0.0,0.0,80.275002,1013.099976,NaN,0.0,0.0,0.0,0.000,16.026501,-10.541000,0.000,273.799988,74.949997,29907.500000,3.950,2.100,3.350,0.0
1,2023-05-01 01:00:00,4.275,1.28600,1482.099976,0.000000e+00,0.000000,1075.100098,0.0,271.450012,0.000000,0.000000,0.000000,0.000000e+00,77.474998,6.0,0.000,0.000,0.000,0.000,0.000,0.00,1.00,1013.150024,0.0,0.0,1000.049988,1006.299988,0.0,0.0,79.824997,1012.599976,NaN,0.0,0.0,0.0,0.000,30.497250,-7.894500,0.000,273.799988,77.474998,29519.074219,3.825,1.925,3.300,0.0
2,2023-05-01 02:00:00,4.150,1.28375,1791.300049,0.000000e+00,0.000000,1200.400024,0.0,271.049988,0.000000,0.000000,0.000000,0.000000e+00,88.099998,6.0,0.000,0.000,0.000,0.000,0.000,0.00,1.00,1012.674988,0.0,0.0,999.500000,1005.799988,0.0,0.0,78.000000,1012.049988,NaN,0.0,0.0,0.0,0.000,44.517250,-3.815500,0.000,273.849976,88.099998,31009.125000,3.650,1.750,3.200,0.0
3,2023-05-01 03:00:00,4.025,1.28200,2312.875000,1.012442e+04,11.675000,1179.849976,0.0,270.649994,9.375000,16845.226562,2.100000,3.765350e+03,68.599998,6.0,0.000,0.000,0.000,0.000,0.000,0.75,0.50,1012.174988,0.0,0.0,998.974976,1005.224976,0.0,0.0,75.625000,1011.525024,NaN,0.0,0.0,0.0,0.000,58.083000,1.412500,0.000,273.899994,68.599998,34552.500000,3.500,1.450,3.150,0.0
4,2023-05-01 04:00:00,3.900,1.28100,2198.299805,1.417486e+05,76.875000,920.049988,0.0,270.375000,47.400002,102209.703125,25.450001,4.957120e+04,66.300003,6.0,0.000,0.000,0.000,0.000,0.000,1.00,0.00,1011.724976,0.0,0.0,998.549988,1004.750000,0.0,0.0,74.224998,1011.049988,NaN,0.0,0.0,0.0,0.000,71.341003,7.468500,0.000,273.924988,66.300003,35483.875000,3.325,1.300,3.050,0.0
5,2023-05-01 05:00:00,4.000,1.27600,1957.625000,4.716072e+05,190.574997,1419.300049,0.0,270.549988,86.425003,240972.250000,88.225006,2.046413e+05,74.074997,6.0,0.000,0.000,0.000,0.000,0.000,1.00,0.00,1011.349976,0.0,0.0,998.125000,1004.349976,0.0,0.0,71.099998,1010.599976,NaN,0.0,0.0,0.0,0.000,84.557755,14.017750,0.000,274.250000,74.074997,35425.449219,3.200,1.400,2.875,0.0
6,2023-05-01 06:00:00,4.000,1.26825,1903.300049,9.212486e+05,322.149994,1299.400024,0.0,270.824982,111.074997,355522.968750,181.500000,4.855580e+05,58.550003,6.0,0.000,0.000,0.000,0.000,0.000,1.00,0.00,1011.000000,0.0,0.0,997.750000,1003.950012,0.0,0.0,66.599998,1010.174988,NaN,0.0,0.0,0.0,0.000,98.091248,20.711750,0.000,274.950012,58.550003,33709.773438,3.550,2.275,2.675,0.0
7,2023-05-01 07:00:00,4.000,1.26250,NaN,1.394139e+06,450.575012,743.299988,0.0,270.899994,130.100006,434118.187500,266.750000,8.068865e+05,48.549999,6.0,0.000,0.000,0.000,0.000,0.000,1.00,0.00,1010.724976,0.0,0.0,997.450012,1003.625000,0.0,0.0,63.375000,1009.849976,NaN,0.0,0.0,0.0,0.000,112.359497,27.171749,0.000,275.575012,48.549999,26871.000000,4.200,3.475,2.350,0.0
8,2023-05-01 08:00:00,4.100,1.25975,1381.199829,1.830129e+06,562.450012,805.375000,0.0,271.024994,159.225006,520811.437500,315.200012,1.047560e+06,61.075001,6.0,0.000,0.000,0.000,0.000,0.000,1.00,0.00,1010.450012

Sample rate 15M --> 1T for observed data

In [13]:
A_observed = Master.averaged(A_observed,"date_forecast")
A_observed = Master.onehot_estimated(A_observed,False)
A_estimated = Master.averaged(A_estimated,"date_forecast")
A_estimated = Master.onehot_estimated(A_estimated,True)

B_observed = Master.averaged(B_observed,"date_forecast")
B_observed = Master.onehot_estimated(B_observed,False)
B_estimated = Master.averaged(B_estimated,"date_forecast")
B_estimated = Master.onehot_estimated(B_estimated,True)


C_observed = Master.averaged(C_observed,"date_forecast")
C_observed = Master.onehot_estimated(C_observed,False)
C_estimated = Master.averaged(C_estimated,"date_forecast")
C_estimated = Master.onehot_estimated(C_estimated,True)



#Do so for test data as well
test_A=Master.averaged_test(test_A,"date_forecast")
test_A=Master.onehot_estimated(test_A,True)
test_A=Master.onehot_location(test_A,"A")

test_B=Master.averaged_test(test_B,"date_forecast")
test_B=Master.onehot_estimated(test_B,True)
test_B=Master.onehot_location(test_B,"B")



test_C=Master.averaged_test(test_C,"date_forecast")
test_C=Master.onehot_estimated(test_C,True)
test_C=Master.onehot_location(test_C,"C")


#C_estimated.head()
print(test_A.shape)
test_A.head()


(720, 51)


/Users/matsalexander/Desktop/Forest Gump/henning mats/Master.py:48: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ['date', 'hour']).mean().reset_index()
/Users/matsalexander/Desktop/Forest Gump/henning mats/Master.py:48: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ['date', 'hour']).mean().reset_index()
/Users/matsalexander/Desktop/Forest Gump/henning mats/Master.py:48: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  [

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,date_forecast,estimated,observed,A,B,C
0,4.325,1.28675,912.700012,0.000000,0.000,1061.550049,0.0,271.650024,0.000000,0.000000,0.000000,0.000000,74.949997,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,1013.674988,0.0,0.0,1000.549988,1006.799988,0.0,0.0,80.275002,1013.099976,NaN,0.0,0.0,0.0,0.0,16.026501,-10.5410,0.0,273.799988,74.949997,29907.500000,3.950,2.100,3.35,0.0,2023-05-01 00:00:00,1,0,1,0,0
1,4.275,1.28600,1482.099976,0.000000,0.000,1075.100098,0.0,271.450012,0.000000,0.000000,0.000000,0.000000,77.474998,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,1013.150024,0.0,0.0,1000.049988,1006.299988,0.0,0.0,79.824997,1012.599976,NaN,0.0,0.0,0.0,0.0,30.497250,-7.8945,0.0,273.799988,77.474998,29519.074219,3.825,1.925,3.30,0.0,2023-05-01 01:00:00,1,0,1,0,0
2,4.150,1.28375,1791.300049,0.000000,0.000,1200.400024,0.0,271.049988,0.000000,0.000000,0.000000,0.000000,88.099998,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,1012.674988,0.0,0.0,999.500000,1005.799988,0.0,0.0,78.000000,1012.049988,NaN,0.0,0.0,0.0,0.0,44.517250,-3.8155,0.0,273.849976,88.099998,31009.125000,3.650,1.750,3.20,0.0,2023-05-01 02:00:00,1,0,1,0,0
3,4.025,1.28200,2312.875000,10124.424805,11.675,1179.849976,0.0,270.649994,9.375000,16845.226562,2.100000,3765.350098,68.599998,6.0,0.0,0.0,0.0,0.0,0.0,0.75,0.5,1012.174988,0.0,0.0,998.974976,1005.224976,0.0,0.0,75.625000,1011.525024,NaN,0.0,0.0,0.0,0.0,58.083000,1.4125,0.0,273.899994,68.599998,34552.500000,3.500,1.450,3.15,0.0,2023-05-01 03:00:00,1,0,1,0,0
4,3.900,1.28100,2198.299805,141748.593750,76.875,920.049988,0.0,270.375000,47.400002,102209.703125,25.450001,49571.199219,66.300003,6.0,0.0,0.0,0.0,0.0,0.0,1.00,0.0,1011.724976,0.0,0.0,998.549988,1004.750000,0.0,0.0,74.224998,1011.049988,NaN,0.0,0.0,0.0,0.0,71.341003,7.4685,0.0,273.924988,66.300003,35483.875000,3.325,1.300,3.05,0.0,2023-05-01 04:00:00,1,0,1,0,0


In [14]:
#A_observed_estimated = Master.target_join(A_observed,A_estimated,target_A)
#B_observed_estimated = Master.target_join(B_observed,B_estimated,target_B)
##C_observed_estimated = Master.target_join(C_observed,C_estimated,target_C)

A_observed_estimated  = pd.concat([A_observed,A_estimated],ignore_index=True)

A_observed_estimated  = Master.onehot_location(A_observed_estimated,"A")

B_observed_estimated  = pd.concat([B_observed,B_estimated],ignore_index=True)
B_observed_estimated = Master.onehot_location(B_observed_estimated,"B")

C_observed_estimated  = pd.concat([C_observed,C_estimated],ignore_index=True)
C_observed_estimated = Master.onehot_location(C_observed_estimated,"C")

#A_observed_estimated=Master.inner_join(A_estimated,target_A)
A_observed_estimated_target= Master.inner_join(A_observed_estimated,target_A)
B_observed_estimated_target = Master.inner_join(B_observed_estimated,target_B)
C_observed_estimated_target = Master.inner_join(C_observed_estimated,target_C)

#Trying only estimated from A



#For just estimated

#A_estimated_only, target_A = Master.inner_join(A_estimated,target_A)
#B_estimated_only, target_B = Master.inner_join(B_estimated,target_B)
#C_estimated_only, target_C = Master.inner_join(C_estimated,target_C)

#For just Observed. 

#A_observed_only, target_A = Master.inner_join(A_observed,target_A)
#B_observed_only, target_B = Master.inner_join(B_observed,target_B)
#C_observed_only, target_C = Master.inner_join(C_observed,target_C)

#A_B_C_est_only=pd.concat([A_estimated_only,B_estimated_only,C_estimated_only])
#A_B_C_obs_only=pd.concat([A_observed_only,B_observed_only,C_observed_only])

#
#A_B_C_all=pd.concat([A_observed_estimated_target,B_observed_estimated_target,C_observed_estimated_target])
#
#A_B_C_est_only.shape

#A_observed_estimated_target.head()






Dt64 feature --> Two features: days_since_jan, hour

In [15]:
A_observed_estimated=Master.timestamp(A_observed_estimated,"date_forecast")
B_observed_estimated=Master.timestamp(B_observed_estimated,"date_forecast")
C_observed_estimated=Master.timestamp(C_observed_estimated,"date_forecast")
test_A=Master.timestamp(test_A,"date_forecast")
test_B=Master.timestamp(test_B,"date_forecast")
test_C=Master.timestamp(test_C,"date_forecast")



A_B_C_all=pd.concat([A_observed_estimated_target,B_observed_estimated_target,C_observed_estimated_target])



# Do so for test set also
#
#test_A=Master.timestamp(test_A,"date_forecast")#
#test_B=Master.timestamp(test_B,"date_forecast")
#test_C=Master.timestamp(test_C,"date_forecast")










Concatinating A,B,C with all accessible values, this is jalla jalla

In [16]:
#A_B_C_obs_est = pd.concat([A_observed_estimated,B_observed_estimated,C_observed_estimated],ignore_index=True)
#target_A_B_C = pd.concat([target_A,target_B,target_C],ignore_index=True).drop(columns=["time"])


test_A_B_C = pd.concat([test_A,test_B,test_C],ignore_index=True)

#uncomment all above for all data

#A_B_C_est_only = pd.concat([A_estimated_only,B_estimated_only,C_estimated_only],ignore_index=True).drop(columns=["date_forecast"])


#A_B_C_est_only.shape
#target_A_B_C.shape
#test_A_B_C.head()
#A_B_C_all.head()
A_B_C_all = Master.timestamp(A_B_C_all,"date_forecast")


In [21]:
A_B_C_all.head(30)

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,estimated,observed,A,B,C,pv_measurement,days_since_jan,hour
0,7.700000,1.22825,1728.949951,0.000000e+00,0.000000,1728.949951,0.0,280.299988,0.000000,0.000000,0.000000,0.000000e+00,99.074997,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.00,1006.299988,0.0000,0.00,993.750000,999.775024,0.0,0.000,71.674995,1005.799988,NaN,0.0,0.0,0.0,0.175,348.036743,-3.774250,0.000,286.225006,100.000000,40386.476562,3.600,-3.575,-0.500,0.0,0,1,1,0,0,0.00,153,22
1,7.700000,1.22350,1689.824951,0.000000e+00,0.000000,1689.824951,0.0,280.299988,0.000000,0.000000,0.000000,0.000000e+00,99.750000,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.00,1005.200012,0.0000,0.00,992.674988,998.650024,0.0,0.025,68.000000,1004.650024,NaN,0.0,0.0,0.0,0.200,91.980751,-4.357250,0.000,286.899994,100.000000,33770.648438,3.350,-3.350,0.275,0.0,0,1,1,0,0,0.00,153,23
2,7.875000,1.21975,1563.224976,0.000000e+00,0.000000,1563.224976,0.0,280.649994,0.000000,0.000000,0.000000,0.000000e+00,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.00,1004.525024,0.0000,0.00,992.000000,997.974976,0.0,0.100,67.949997,1003.950012,NaN,0.0,0.0,0.0,0.400,14.934750,-3.309500,0.000,286.950012,100.000000,13595.500000,3.050,-2.950,0.750,0.0,0,1,1,0,0,0.00,154,0
3,8.425000,1.21800,1283.425049,2.086500e+02,0.750000,1283.425049,0.0,281.674988,0.300000,526.775024,0.000000,0.000000e+00,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.25,1.00,1004.025024,0.0000,0.00,991.500000,997.449951,0.0,0.125,73.875000,1003.449951,NaN,0.0,0.0,0.0,0.550,28.630251,-0.822500,0.000,286.750000,100.000000,2321.850098,2.725,-2.600,0.875,0.0,0,1,1,0,0,0.00,154,1
4,8.950000,1.21800,1003.500000,3.246815e+04,23.100000,1003.500000,0.0,282.500000,11.975000,22068.949219,0.150000,2.829750e+02,84.875000,6.0,0.0,0.0,0.0,0.0,0.0,1.00,0.00,1003.099976,0.0000,0.00,990.550049,996.500000,0.0,0.100,79.925003,1002.500000,NaN,0.0,0.0,0.0,0.250,41.997501,3.051250,0.000,286.450012,99.224998,11634.799805,2.550,-2.350,0.925,0.0,0,1,1,0,0,19.36,154,2
5,9.250000,1.21650,809.375000,1.794991e+05,84.375000,809.375000,0.0,283.049988,45.125000,102788.046875,6.300000,1.162755e+04,65.025002,6.0,0.0,0.0,0.0,0.0,0.0,1.00,0.00,1002.349976,0.0000,0.00,989.750000,995.700012,0.0,0.000,82.849998,1001.674988,NaN,0.0,0.0,0.0,0.100,55.041748,8.071000,0.000,286.475006,94.800003,29848.199219,2.300,-2.200,0.800,0.0,0,1,1,0,0,251.02,154,3
6,9.525000,1.21300,757.775024,4.781178e+05,186.649994,757.775024,0.0,283.524994,89.525002,242438.000000,43.724998,9.007338e+04,77.550003,6.0,0.0,0.0,0.0,0.0,0.0,1.00,0.00,1001.825012,0.0000,0.00,989.250000,995.200012,0.0,0.000,82.275002,1001.174988,NaN,0.0,0.0,0.0,0.000,67.898003,13.956500,0.000,286.825012,95.099998,35980.148438,2.325,-2.200,0.750,0.0,0,1,1,0,0,263.78,154,4
7,9.700000,1.20750,705.650024,8.926679e+05,311.525024,705.650024,0.0,283.799988,139.000000,411381.906250,53.099998,1.742545e+05,85.050003,6.0,0.0,0.0,0.0,0.0,0.0,1.00,0.00,1001.200012,0.0000,0.00,988.700012,994.599976,0.0,0.000,78.699997,1000.500000,NaN,0.0,0.0,0.0,0.000,80.821251,20.406250,0.000,287.325012,97.150002,37874.523438,2.225,-2.000,0.950,0.0,0,1,1,0,0,522.72,154,5
8,9.550000,1.20500,669.650024,1.357902e+06,442.750000,669.650024,0.0,283.600006,167.100006,550960.562500,33.825001,1.564732e+05,96.750000,6.0,0.0,0.0,0.0,0.0,0

In [7]:
test_A_B_C.head()

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,estimated,observed,A,B,C,days_since_jan,hour
0,4.325,1.28675,912.700012,0.000000,0.000,1061.550049,0.0,271.650024,0.000000,0.000000,0.000000,0.000000,74.949997,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,1013.674988,0.0,0.0,1000.549988,1006.799988,0.0,0.0,80.275002,1013.099976,NaN,0.0,0.0,0.0,0.0,16.026501,-10.5410,0.0,273.799988,74.949997,29907.500000,3.950,2.100,3.35,0.0,1,0,1,0,0,121,0
1,4.275,1.28600,1482.099976,0.000000,0.000,1075.100098,0.0,271.450012,0.000000,0.000000,0.000000,0.000000,77.474998,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,1013.150024,0.0,0.0,1000.049988,1006.299988,0.0,0.0,79.824997,1012.599976,NaN,0.0,0.0,0.0,0.0,30.497250,-7.8945,0.0,273.799988,77.474998,29519.074219,3.825,1.925,3.30,0.0,1,0,1,0,0,121,1
2,4.150,1.28375,1791.300049,0.000000,0.000,1200.400024,0.0,271.049988,0.000000,0.000000,0.000000,0.000000,88.099998,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,1012.674988,0.0,0.0,999.500000,1005.799988,0.0,0.0,78.000000,1012.049988,NaN,0.0,0.0,0.0,0.0,44.517250,-3.8155,0.0,273.849976,88.099998,31009.125000,3.650,1.750,3.20,0.0,1,0,1,0,0,121,2
3,4.025,1.28200,2312.875000,10124.424805,11.675,1179.849976,0.0,270.649994,9.375000,16845.226562,2.100000,3765.350098,68.599998,6.0,0.0,0.0,0.0,0.0,0.0,0.75,0.5,1012.174988,0.0,0.0,998.974976,1005.224976,0.0,0.0,75.625000,1011.525024,NaN,0.0,0.0,0.0,0.0,58.083000,1.4125,0.0,273.899994,68.599998,34552.500000,3.500,1.450,3.15,0.0,1,0,1,0,0,121,3
4,3.900,1.28100,2198.299805,141748.593750,76.875,920.049988,0.0,270.375000,47.400002,102209.703125,25.450001,49571.199219,66.300003,6.0,0.0,0.0,0.0,0.0,0.0,1.00,0.0,1011.724976,0.0,0.0,998.549988,1004.750000,0.0,0.0,74.224998,1011.049988,NaN,0.0,0.0,0.0,0.0,71.341003,7.4685,0.0,273.924988,66.300003,35483.875000,3.325,1.300,3.05,0.0,1,0,1,0,0,121,4


In [8]:

#X_train=A_B_C_est_only
#X_train=A_B_C_obs_only
#X_train=A_B_C_all
#Y_train = target_A_B_C

train_data = TabularDataset(A_B_C_all)

#train_A=TabularDataset(A_observed_estimated_target)
#train_B=TabularDataset(B_observed_estimated_target)
#train_C=TabularDataset(C_observed_estimated_target)

#train_A=Master.timestamp(A_observed_estimated_target,"date_forecast")
#train_B=Master.timestamp(B_observed_estimated_target,"date_forecast")
#train_C=Master.timestamp(C_observed_estimated_target,"date_forecast")

#train_data.fillna(0)

#test_data = TabularDataset(test_A_B_C)
#test_data.fillna(0)
#to_be_predicted = test_A_B_C

label="pv_measurement"

#train_data[label].describe()

#num_bag_folds = 5  # Using 5 bagging folds as an example
#num_stack_levels = 2  # Using 2 stacking levels as an example

# Initialize the predictor and fit with bagging and stacking enabled
#predictor = TabularPredictor(label=label).fit(train_data, num_bag_folds=num_bag_folds, num_stack_levels=num_stack_levels)

predictor = TabularPredictor(label=label,eval_metric='mean_absolute_error').fit(A_B_C_all,presets="best_quality")




#Need to do some fillin

#X_train["snow_density:kgm3"].fillna(0, inplace=True)
#X_train = X_train.interpolate().fillna(0)

#to_be_predicted["snow_density:kgm3"].fillna(0, inplace=True)
#to_be_predicted = to_be_predicted.interpolate().fillna(0)

#to_be_predicted.fillna(0, inplace=True)

#null_counts = Y_train.isnull().sum()
#print(null_counts)

# Assuming X and y are your data and labels, respectively
#model = LightGMB()
#model.train(X_train, Y_train)

# Assuming unlabeled_df is your DataFrame with unlabeled data
#predictions = model.predict(to_be_predicted)

#predictions_df = pd.DataFrame(predictions, columns=['predictions'])  # adjust column name as needed
#predictions_df[predictions_df < 0] = 0
#predictions_df['id'] = range(len(predictions_df))
#predictions_df.set_index('id', inplace=True)
#predictions_df.to_csv('predictions.csv', index=True)



No path specified. Models will be saved in: "AutogluonModels/ag-20231106_094958/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231106_094958/"
AutoGluon Version:  0.8.2
Python Version:     3.10.13
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 22.5.0: Thu Jun  8 22:22:20 PDT 2023; root:xnu-8796.121.3~7/RELEASE_ARM64_T6000
Disk Space Avail:   735.11 GB / 994.66 GB (73.9%)
Train Data Rows:    93023
Train Data Columns: 52
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (5733.42, -0.0, 287.02274, 766.41133)
	If 'regression' is not the correct problem_type, please manually specify the problem_type param

In [9]:
#predictor_B = TabularPredictor(label=label,eval_metric='mean_absolute_error').fit(train_B,presets="best_quality")

In [10]:
#predictor_C = TabularPredictor(label=label,eval_metric='mean_absolute_error').fit(train_C,presets="best_quality")

In [11]:
#predictions_A = predictor_A.predict(test_A)
#predictions_B = predictor_B.predict(test_B)
#predictions_C = predictor_C.predict(test_C)

predictions=predictor.predict(test_A_B_C)

In [12]:
#predictions = pd.concat([predictions_A, predictions_B, predictions_C], ignore_index=True)



In [13]:
#predictions = np.maximum(predictions, 0)

predictions_df = pd.DataFrame({
    'prediction': predictions
})

# Add the 'id' column starting from 0 for 0-indexing
predictions_df.insert(0, 'id', range(0, len(predictions_df)))

#predictions_df['prediction'] = np.where(predictions_df['prediction'] < 0, 0, predictions_df['prediction'])

predictions_df.head()




,id,prediction
0,0,0.080075
1,1,0.035356
2,2,0.150903
3,3,60.781342
4,4,338.730713


In [14]:
predictions_df['prediction'] = pd.to_numeric(predictions_df['prediction'], errors='coerce')

predictions_df['prediction'] = predictions_df['prediction'].clip(lower=0)

predictions_df.to_csv('predictions_auto.csv', index=False)


Now for the grand finale of trying it on test data, jalla jalla for now


X_train = A_B_C_obs_est
Y_train = target_A_B_C

to_be_predicted = test_A_B_C

X_train.fillna(0, inplace=True)
to_be_predicted.fillna(0, inplace=True)

null_counts = Y_train.isnull().sum()
print(null_counts)

base_models = [('random_forest', RandomForestRegressor(n_estimators=100, random_state=42))]

# Use XGBoost as the meta-model
#meta_model = XGBRegressor(n_estimators=100, random_state=42)

# Create and fit the stacking regressor
#stacking_regressor = StackingRegressor(estimators=base_models, final_estimator=meta_model)
#stacking_regressor.fit(X_train, Y_train)





model = RandomForestRegressor(n_estimators=1000, random_state=42)  # Create a Random Forest Regressor model
model.fit(X_train, Y_train)  # Train the model with the training data
predictions = model.predict(to_be_predicted)
#predictions = stacking_regressor.predict(to_be_predicted)
predictions_df = pd.DataFrame(predictions, columns=['predictions'])  # adjust column name as needed
predictions_df['id'] = range(len(predictions_df))
predictions_df.set_index('id', inplace=True)
predictions_df.to_csv('predictions.csv', index=True)




In [15]:
test_A_B_C.head()

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,estimated,observed,A,B,C,days_since_jan,hour
0,4.325,1.28675,912.700012,0.000000,0.000,1061.550049,0.0,271.650024,0.000000,0.000000,0.000000,0.000000,74.949997,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,1013.674988,0.0,0.0,1000.549988,1006.799988,0.0,0.0,80.275002,1013.099976,NaN,0.0,0.0,0.0,0.0,16.026501,-10.5410,0.0,273.799988,74.949997,29907.500000,3.950,2.100,3.35,0.0,1,0,1,0,0,121,0
1,4.275,1.28600,1482.099976,0.000000,0.000,1075.100098,0.0,271.450012,0.000000,0.000000,0.000000,0.000000,77.474998,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,1013.150024,0.0,0.0,1000.049988,1006.299988,0.0,0.0,79.824997,1012.599976,NaN,0.0,0.0,0.0,0.0,30.497250,-7.8945,0.0,273.799988,77.474998,29519.074219,3.825,1.925,3.30,0.0,1,0,1,0,0,121,1
2,4.150,1.28375,1791.300049,0.000000,0.000,1200.400024,0.0,271.049988,0.000000,0.000000,0.000000,0.000000,88.099998,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,1012.674988,0.0,0.0,999.500000,1005.799988,0.0,0.0,78.000000,1012.049988,NaN,0.0,0.0,0.0,0.0,44.517250,-3.8155,0.0,273.849976,88.099998,31009.125000,3.650,1.750,3.20,0.0,1,0,1,0,0,121,2
3,4.025,1.28200,2312.875000,10124.424805,11.675,1179.849976,0.0,270.649994,9.375000,16845.226562,2.100000,3765.350098,68.599998,6.0,0.0,0.0,0.0,0.0,0.0,0.75,0.5,1012.174988,0.0,0.0,998.974976,1005.224976,0.0,0.0,75.625000,1011.525024,NaN,0.0,0.0,0.0,0.0,58.083000,1.4125,0.0,273.899994,68.599998,34552.500000,3.500,1.450,3.15,0.0,1,0,1,0,0,121,3
4,3.900,1.28100,2198.299805,141748.593750,76.875,920.049988,0.0,270.375000,47.400002,102209.703125,25.450001,49571.199219,66.300003,6.0,0.0,0.0,0.0,0.0,0.0,1.00,0.0,1011.724976,0.0,0.0,998.549988,1004.750000,0.0,0.0,74.224998,1011.049988,NaN,0.0,0.0,0.0,0.0,71.341003,7.4685,0.0,273.924988,66.300003,35483.875000,3.325,1.300,3.05,0.0,1,0,1,0,0,121,4
